<a href="https://colab.research.google.com/github/Hanbin-git/DNA/blob/main/data_finetuning_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U bitsandbytes peft accelerate transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompat

In [3]:
# 압축풀기
import zipfile
import os

zip_path = "/open.zip"
extract_dir = "/content/open"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ 압축 해제 완료!")
print("압축 풀린 파일 목록:", os.listdir(extract_dir))

✅ 압축 해제 완료!
압축 풀린 파일 목록: ['test.csv', 'sample_submission.csv']


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 데이터 불러오기
import pandas as pd

data_dir = "/content/open"

test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))
sub_df  = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

print("✅ test_df:", test_df.shape)
print("✅ sub_df:", sub_df.shape)
test_df.head()


✅ test_df: (13711, 2)
✅ sub_df: (13711, 769)


,ID,seq
0,TEST_000000,ATCATTTTTATTTTTTAGTTTTATGAGACGCTGCCTTGCTATGTCA...
1,TEST_000001,CGACGTCCCCGTAGCGGCCGAAGTCGAGGGGCAGCAGGCGATCGTG...
2,TEST_000002,GGTAGTAAGAAGGAAAATGACAGCATGGAAGCAGCAATACCAGTAA...
3,TEST_000003,CAGCGCATATACTCAGGGCCATGGTGGGTACTGTTCCCATGGCCAG...
4,TEST_000004,TTCATAATTGCTATCAGTCTATGGGCTAATATTTTATACATCAATG...


In [5]:
import os, math, random, zipfile
from collections import Counter
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

# 재현성
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

# 환경 변수
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", DEVICE)

# 압축 해제
def extract_open_zip():
    candidates = ["/content/drive/MyDrive/DNA/open.zip", "/content/open.zip", "/open.zip"]
    for c in candidates:
        if os.path.exists(c):
            zip_path = c; break
    else:
        raise FileNotFoundError("open.zip 경로를 찾을 수 없습니다.")
    extract_dir = "/content/open"
    os.makedirs(extract_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_dir)
    print("✅ 압축 해제 완료:", os.listdir(extract_dir))
    return extract_dir

data_dir = extract_open_zip()
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))
sub_df  = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
print("✅ test_df:", test_df.shape)
display(test_df.head())


✅ Device: cuda
✅ 압축 해제 완료: ['test.csv', 'sample_submission.csv']
✅ test_df: (13711, 2)


,ID,seq
0,TEST_000000,ATCATTTTTATTTTTTAGTTTTATGAGACGCTGCCTTGCTATGTCA...
1,TEST_000001,CGACGTCCCCGTAGCGGCCGAAGTCGAGGGGCAGCAGGCGATCGTG...
2,TEST_000002,GGTAGTAAGAAGGAAAATGACAGCATGGAAGCAGCAATACCAGTAA...
3,TEST_000003,CAGCGCATATACTCAGGGCCATGGTGGGTACTGTTCCCATGGCCAG...
4,TEST_000004,TTCATAATTGCTATCAGTCTATGGGCTAATATTTTATACATCAATG...


In [6]:
def gc_content(seq: str) -> float:
    seq = seq.upper()
    return (seq.count("G") + seq.count("C")) / len(seq) if len(seq) > 0 else 0.0

def seq_entropy(seq: str) -> float:
    c = Counter(seq)
    n = len(seq)
    return 0.0 if n == 0 else -sum((v/n)*math.log2(v/n) for v in c.values())

corpus_df = test_df.copy()
corpus_df["gc"] = corpus_df["seq"].apply(gc_content)
corpus_df["entropy"] = corpus_df["seq"].apply(seq_entropy)
ENT_MIN, ENT_MAX = corpus_df["entropy"].min(), corpus_df["entropy"].max()
print("✅ 파생변수 생성 완료")


✅ 파생변수 생성 완료


In [7]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForMaskedLM, BitsAndBytesConfig

MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"

# 8-bit 로딩
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map={"": 0},
)
model.config.use_cache = False

# ✅ MLM용: FEATURE_EXTRACTION 사용
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=[
        "query","key","value","dense",          # attention
        "intermediate.dense","output.dense"     # FFN (있으면 자동 적용, 없으면 무시)
    ],
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION,      # ← 여기!
)

model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/nucleotide-transformer-v2-500m-multi-species:
- esm_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_esm.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/nucleotide-transformer-v2-500m-multi-species:
- modeling_esm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

trainable params: 5,242,880 || all params: 503,588,316 || trainable%: 1.0411


In [8]:
# 1개 시퀀스로 더미 전/후 처리 테스트
import torch
tmp = tokenizer("ACGT"*256, return_tensors="pt", truncation=True, max_length=512)
with torch.no_grad():
    _ = model(**{k: v.to(model.device) for k,v in tmp.items()})
print("✅ forward ok")


✅ forward ok


In [10]:
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
from transformers import get_linear_schedule_with_warmup

class MLMDataset(Dataset):
    def __init__(self, df): self.df = df
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        x = self.df.iloc[i]
        return {"seq": x.seq, "gc": x.gc, "entropy": x.entropy}

def mask_prob_from_gc_entropy(gc, entropy, base=0.15, gc_center=0.5, gc_w=0.10, ent_w=0.05):
    gc_factor = 1.0 - abs(gc - gc_center)
    ent_norm = (entropy - ENT_MIN) / (ENT_MAX - ENT_MIN + 1e-8)
    return float(np.clip(base + gc_w*gc_factor + ent_w*ent_norm, 0.05, 0.40))

class VariantAwareMLMCollator:
    def __init__(self, tok, max_len=1024):
        self.tok, self.max_len = tok, max_len
        self.pad_id, self.mask_id = tok.pad_token_id, tok.mask_token_id
        self.vocab_size = tok.vocab_size
        self.special_ids = set(tok.all_special_ids)

    def __call__(self, batch):
        seqs = [b["seq"] for b in batch]
        gcs  = [b["gc"] for b in batch]
        ents = [b["entropy"] for b in batch]
        enc = self.tok.batch_encode_plus(seqs, padding="longest", truncation=True,
                                         max_length=self.max_len, return_tensors="pt")
        input_ids, attn = enc["input_ids"], enc["attention_mask"]
        labels = input_ids.clone()
        B, L = input_ids.shape
        rand = torch.rand((B, L))
        probs = torch.tensor([mask_prob_from_gc_entropy(gc, ent) for gc, ent in zip(gcs, ents)]).unsqueeze(1)
        mask_cand = (rand < probs) & (input_ids != self.pad_id)
        for sid in self.special_ids: mask_cand &= (input_ids != sid)
        labels[~mask_cand] = -100
        mask80 = mask_cand & (torch.rand((B, L)) < 0.8)
        input_ids[mask80] = self.mask_id
        mask10 = mask_cand & (~mask80) & (torch.rand((B, L)) < 0.5)
        rand_tokens = torch.randint(self.vocab_size, (B, L))
        input_ids[mask10] = rand_tokens[mask10]
        return {"input_ids": input_ids, "attention_mask": attn, "labels": labels}

# Dataloader
mlm_loader = DataLoader(MLMDataset(corpus_df), batch_size=16, shuffle=True,
                        collate_fn=VariantAwareMLMCollator(tokenizer, max_len=1024))

optim = torch.optim.AdamW(model.parameters(), lr=3e-5)
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=100, num_training_steps=len(mlm_loader))
scaler = GradScaler("cuda")

ACCUM = 64
model.train()
for step, batch in enumerate(mlm_loader, 1):
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    with autocast(device_type="cuda", dtype=torch.float16):
        out = model(**batch)
        loss = out.loss / ACCUM
    scaler.scale(loss).backward()
    if step % ACCUM == 0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optim); scaler.update()
        optim.zero_grad(set_to_none=True); scheduler.step()
    if step % 100 == 0:
        print(f"[S1] Step {step}/{len(mlm_loader)} Loss: {loss.item()*ACCUM:.4f}")

stage1_ckpt = "/content/stage1_lora8bit"
os.makedirs(stage1_ckpt, exist_ok=True)
model.save_pretrained(stage1_ckpt); tokenizer.save_pretrained(stage1_ckpt)
print("✅ Stage 1 저장 완료:", stage1_ckpt)
torch.cuda.empty_cache()


[S1] Step 100/857 Loss: 7.0289
[S1] Step 200/857 Loss: 7.9267
[S1] Step 300/857 Loss: 7.3676
[S1] Step 400/857 Loss: 7.3167
[S1] Step 500/857 Loss: 7.9190
[S1] Step 600/857 Loss: 7.1287
[S1] Step 700/857 Loss: 7.3484
[S1] Step 800/857 Loss: 7.9415
✅ Stage 1 저장 완료: /content/stage1_lora8bit


In [11]:
BASES = ["A","C","G","T"]
def random_snv(seq, num_mut=1):
    s = list(seq)
    for pos in random.sample(range(len(s)), min(num_mut, len(s))):
        cand = [b for b in BASES if b != s[pos]]
        s[pos] = random.choice(cand)
    return "".join(s)

class ContrastiveDataset(Dataset):
    def __init__(self, df): self.df = df
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        seq = self.df.iloc[i].seq
        return {"seq": seq, "seq_view": seq, "seq_mut": random_snv(seq, 1)}

def collate_contrastive(batch, tok=tokenizer, max_len=1024):
    def enc(xs):
        e = tok.batch_encode_plus(xs, padding="longest", truncation=True, max_length=max_len, return_tensors="pt")
        return e["input_ids"], e["attention_mask"]
    seqs = [b["seq"] for b in batch]
    seqs_view = [b["seq_view"] for b in batch]
    seqs_mut = [b["seq_mut"] for b in batch]
    oi, om = enc(seqs); vi, vm = enc(seqs_view); mi, mm = enc(seqs_mut)
    return {"oi":oi,"om":om,"vi":vi,"vm":vm,"mi":mi,"mm":mm}

from torch.nn.functional import cosine_similarity
def mean_pool_last_hidden(outputs, mask):
    hs = outputs.hidden_states[-1]
    m = mask.unsqueeze(-1)
    return (hs*m).sum(1)/m.sum(1).clamp(min=1)

model = AutoModelForMaskedLM.from_pretrained(stage1_ckpt, trust_remote_code=True,
                                             quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_cfg)
model.train()

ctr_loader = DataLoader(ContrastiveDataset(corpus_df), batch_size=4, shuffle=True,
                        collate_fn=lambda b: collate_contrastive(b, tok=tokenizer))

optim2 = torch.optim.AdamW(model.parameters(), lr=2e-5)
scaler2 = GradScaler("cuda")
ACCUM2, MARGIN = 64, 0.1

for step, batch in enumerate(ctr_loader, 1):
    oi, om = batch["oi"].to(DEVICE), batch["om"].to(DEVICE)
    vi, vm = batch["vi"].to(DEVICE), batch["vm"].to(DEVICE)
    mi, mm = batch["mi"].to(DEVICE), batch["mm"].to(DEVICE)
    with autocast(device_type="cuda", dtype=torch.float16):
        zo = mean_pool_last_hidden(model(oi, attention_mask=om, output_hidden_states=True), om)
        zv = mean_pool_last_hidden(model(vi, attention_mask=vm, output_hidden_states=True), vm).detach()
        zm = mean_pool_last_hidden(model(mi, attention_mask=mm, output_hidden_states=True), mm).detach()
        sim_pos = cosine_similarity(zo, zv)
        sim_neg = cosine_similarity(zo, zm)
        loss = torch.clamp(MARGIN + sim_neg - sim_pos, min=0).mean() / ACCUM2
    scaler2.scale(loss).backward()
    if step % ACCUM2 == 0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler2.step(optim2); scaler2.update(); optim2.zero_grad(set_to_none=True)
    if step % 100 == 0:
        print(f"[S2] Step {step} loss: {loss.item()*ACCUM2:.4f}")
torch.cuda.empty_cache()

stage2_ckpt = "/content/stage2_contrastive_lora8bit"
os.makedirs(stage2_ckpt, exist_ok=True)
model.save_pretrained(stage2_ckpt); tokenizer.save_pretrained(stage2_ckpt)
print("✅ Stage 2 저장 완료:", stage2_ckpt)


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


[S2] Step 100 loss: 0.0995


KeyboardInterrupt: 

In [17]:
from torch.utils.data import Dataset, DataLoader

class SeqDataset(Dataset):
    def __init__(self, df): self.ids, self.seqs = df["ID"].tolist(), df["seq"].tolist()
    def __len__(self): return len(self.ids)
    def __getitem__(self, i): return {"ID": self.ids[i], "seq": self.seqs[i]}

def collate_infer(batch, tok=tokenizer, max_len=1024):
    ids  = [b["ID"] for b in batch]; seqs = [b["seq"] for b in batch]
    enc = tok.batch_encode_plus(seqs, padding="longest", truncation=True,
                                max_length=max_len, return_tensors="pt")
    return {"ids": ids, "input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"]}

infer_loader = DataLoader(SeqDataset(test_df), batch_size=16, shuffle=False, collate_fn=collate_infer)

@torch.no_grad()
def extract_embeddings(model, loader):
    all_ids, embs = [], []
    for batch in tqdm(loader, desc="Extract"):
        ids = batch["ids"]
        input_ids, attn = batch["input_ids"].to(DEVICE), batch["attention_mask"].to(DEVICE)
        with autocast(device_type="cuda", dtype=torch.float16):
            out = model(input_ids, attention_mask=attn, output_hidden_states=True)
            hs = out.hidden_states[-1]; mask = attn.unsqueeze(-1)
            emb = (hs*mask).sum(1)/mask.sum(1).clamp(min=1)
        all_ids.extend(ids); embs.append(emb.cpu())
        del out, emb; torch.cuda.empty_cache()
    return all_ids, torch.vstack(embs)

model.eval()
ids, emb_tensor = extract_embeddings(model, infer_loader)
H = emb_tensor.shape[1]
emb_df = pd.DataFrame(emb_tensor.numpy(), columns=[f"emb_{i:04d}" for i in range(H)])
submission = pd.concat([sub_df[["ID"]], emb_df], axis=1)
out_path = "/content/submission_stage2_lora8bit.csv"
submission.to_csv(out_path, index=False)
print("✅ 제출 파일 생성 완료:", out_path)
submission.head()


Extract: 100%|██████████| 3428/3428 [13:47<00:00,  4.14it/s]


✅ 제출 파일 생성 완료: /content/submission_stage2_lora8bit.csv


,ID,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_1014,emb_1015,emb_1016,emb_1017,emb_1018,emb_1019,emb_1020,emb_1021,emb_1022,emb_1023
0,TEST_000000,0.139317,0.103059,0.046083,-0.075317,-0.019991,0.040967,-0.138024,-0.802797,0.184288,...,0.150061,0.111468,0.070707,-0.075534,-0.110732,0.017242,0.158625,-0.094519,0.248387,0.101039
1,TEST_000001,-0.012288,0.080854,-0.135634,0.053765,-0.184574,0.244436,0.075660,-0.239170,0.157061,...,-0.050305,0.035269,-0.126393,-0.132549,-0.168340,0.060516,-0.060063,-0.189517,0.623045,-0.149989
2,TEST_000002,0.175488,0.049647,-0.032940,-0.117441,0.001211,0.147004,-0.281818,-0.675481,0.139306,...,0.051925,-0.037222,0.080485,-0.000920,-0.097887,0.085249,0.187503,0.043505,0.153992,0.066804
3,TEST_000003,0.099347,0.076956,0.012087,-0.184017,-0.111378,0.236539,-0.221122,-0.803930,0.049654,...,0.120032,0.098271,0.132732,-0.083911,-0.097768,0.037088,0.159692,0.018994,0.142375,-0.004306
4,TEST_000004,-0.018580,0.064592,-0.106436,-0.075716,-0.026625,0.093453,-0.300043,-0.765590,0.108570,...,0.083171,-0.020589,-0.038171,-0.036910,-0.012016,0.075029,0.050192,-0.101168,0.129030,0.013768


In [ ]:
from google.colab import files
files.download("/content/submission_stage2_lora8bit.csv")


In [12]:
# ============================================
#  제2회 MAI 대회 최종 제출 Inference 파이프라인
#  Author: Hanbin (GPT-5 assisted)
#  Strategy: Weighted last4 mean + Multi-crop + Reverse complement + Whitening
# ============================================

import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.decomposition import PCA
from tqdm import tqdm

# ---------------------------------------------------
# 1️⃣ 환경 설정
# ---------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
print(f"✅ Device: {DEVICE}")

# ---------------------------------------------------
# 2️⃣ 데이터 로드
# ---------------------------------------------------
data_path = "/content/open"
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
sub_df  = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
print("✅ Loaded:", test_df.shape)

# ---------------------------------------------------
# 3️⃣ Reverse Complement 생성 함수
# ---------------------------------------------------
def reverse_complement(seq: str) -> str:
    tr = str.maketrans("ACGT", "TGCA")
    return seq.translate(tr)[::-1]

# ---------------------------------------------------
# 4️⃣ 모델 및 토크나이저 로드
# ---------------------------------------------------
MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(MODEL_ID, trust_remote_code=True)
model = model.to(DEVICE).eval()
model.config.use_cache = False

print("✅ Model loaded.")

# ---------------------------------------------------
# 5️⃣ Multi-crop + Weighted layer embedding 함수
# ---------------------------------------------------
@torch.no_grad()
def get_seq_embedding(seq, model, tokenizer, n_views=6, max_len=1024):
    """랜덤 crop + reverse complement 평균 포함"""
    embs = []
    seqs = [seq, reverse_complement(seq)]  # 원본 + 역상보
    layer_weights = torch.tensor([0.1, 0.2, 0.3, 0.4]).to(DEVICE)

    for s in seqs:
        for _ in range(n_views):
            offset = np.random.randint(0, max(1, len(s) - max_len))
            sub_seq = s[offset:offset + max_len]
            tok = tokenizer(
                sub_seq, return_tensors="pt", truncation=True, padding=True, max_length=max_len
            ).to(DEVICE)
            out = model(**tok, output_hidden_states=True)
            hs = torch.stack(out.hidden_states[-4:], dim=0)  # (4, B, L, H)
            weighted = (hs * layer_weights.view(4,1,1,1)).sum(0)
            mask = tok["attention_mask"].unsqueeze(-1)
            emb = (weighted * mask).sum(1) / mask.sum(1).clamp(min=1)
            embs.append(emb.cpu())

    return torch.stack(embs).mean(0)  # 모든 view 평균

# ---------------------------------------------------
# 6️⃣ 전체 배치 임베딩 추출
# ---------------------------------------------------
all_ids = []
all_embs = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    emb = get_seq_embedding(row["seq"], model, tokenizer, n_views=6, max_len=1024)
    all_ids.append(row["ID"])
    all_embs.append(emb)

emb_tensor = torch.vstack(all_embs)
print("✅ Embedding tensor shape:", emb_tensor.shape)

# ---------------------------------------------------
# 7️⃣ LayerNorm + PCA Whitening
# ---------------------------------------------------
from torch.nn.functional import layer_norm

emb_normed = layer_norm(emb_tensor, emb_tensor.shape[1:])
emb_np = emb_normed.numpy()

# PCA Whitening (512차원 축소 + 정규화)
pca = PCA(n_components=min(512, emb_np.shape[1]), whiten=True, random_state=SEED)
emb_pca = pca.fit_transform(emb_np)

# 다시 정규화
emb_final = emb_pca / np.linalg.norm(emb_pca, axis=1, keepdims=True)
print("✅ Final embedding shape:", emb_final.shape)

# ---------------------------------------------------
# 8️⃣ 제출 파일 생성
# ---------------------------------------------------
emb_cols = [f"emb_{i:04d}" for i in range(emb_final.shape[1])]
emb_df = pd.DataFrame(emb_final, columns=emb_cols)
submission = pd.concat([pd.Series(all_ids, name="ID"), emb_df], axis=1)

out_path = "/content/submission_final_gLM.csv"
submission.to_csv(out_path, index=False)
print("✅ Saved submission file:", out_path)

# ---------------------------------------------------
# 9️⃣ 로컬 다운로드
# ---------------------------------------------------
from google.colab import files
files.download(out_path)


✅ Device: cuda
✅ Loaded: (13711, 2)
✅ Model loaded.


  1%|          | 74/13711 [00:29<1:30:06,  2.52it/s]


KeyboardInterrupt: 

In [ ]:
# 위의 코드 출력 두배로 늘린 것
# ============================================
#  제2회 MAI 대회 최종 제출 Inference 파이프라인 (GPU 활용 강화판)
#  Author: Hanbin (GPT-5 assisted)
#  Strategy: Weighted last4 mean + Multi-crop(batch) + Reverse complement + Whitening
#  Changes for 2x GPU util:
#   - Batch all (n_views * 2) crops per sequence in one forward
#   - AMP (bf16/FP16) autocast
#   - Enable TF32
#   - Try torch.compile for fused kernels (PyTorch 2.x)
# ============================================

import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.decomposition import PCA
from tqdm import tqdm

# ---------------------------------------------------
# 1️⃣ 환경 설정
# ---------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
print(f"✅ Device: {DEVICE}")

# GPU/커널 최적화 스위치
if DEVICE == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True  # TF32 on (Ampere+)
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

def _amp_dtype():
    # bf16 지원(AMPERE+)이면 bf16, 아니면 fp16 사용
    if DEVICE == "cuda" and torch.cuda.is_available():
        cap = torch.cuda.get_device_capability()
        # Ampere(8.0+) 이상이면 bf16 안정
        if cap[0] >= 8:
            return torch.bfloat16
        else:
            return torch.float16
    return None

AMP_DTYPE = _amp_dtype()

# ---------------------------------------------------
# 2️⃣ 데이터 로드
# ---------------------------------------------------
data_path = "/content/open"
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
sub_df  = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
print("✅ Loaded:", test_df.shape)

# ---------------------------------------------------
# 3️⃣ Reverse Complement 생성 함수
# ---------------------------------------------------
def reverse_complement(seq: str) -> str:
    tr = str.maketrans("ACGT", "TGCA")
    return seq.translate(tr)[::-1]

# ---------------------------------------------------
# 4️⃣ 모델 및 토크나이저 로드
# ---------------------------------------------------
MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

# AMP-friendly dtype으로 로드 시도 (메모리 절약 + 속도)
model = AutoModelForMaskedLM.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    torch_dtype=(AMP_DTYPE if AMP_DTYPE is not None else None)
)

model = model.to(DEVICE).eval()
model.config.use_cache = False

# PyTorch 2.x에서 compile 시도 (안되면 그냥 패스)
try:
    model = torch.compile(model, mode="max-autotune")
    print("✅ torch.compile: enabled")
except Exception as e:
    print("⚠️ torch.compile failed, continue without it:", str(e))

print("✅ Model loaded.")

# ---------------------------------------------------
# 5️⃣ Multi-crop (배치 전송) + Weighted layer embedding
#    👉 view들을 한 번에 토크나이즈/전송하여 GPU 활용도 2배↑
# ---------------------------------------------------
@torch.no_grad()
def get_seq_embedding(seq, model, tokenizer, n_views=6, max_len=1024):
    """
    - 원본 + 역상보 각각 n_views개를 한 번의 forward로 처리 (총 2*n_views)
    - AMP(bf16/fp16)로 throughput 향상
    - 마지막 4개 히든스테이트 가중합(Weighted last4)
    """
    # 시퀀스 후보 생성
    srcs = [seq, reverse_complement(seq)]
    subseqs = []
    for s in srcs:
        L = len(s)
        # L < max_len인 경우 offset=0 고정
        for _ in range(n_views):
            if L <= max_len:
                sub_seq = s
            else:
                offset = np.random.randint(0, L - max_len + 1)
                sub_seq = s[offset:offset + max_len]
            subseqs.append(sub_seq)

    # 토크나이즈 (배치)
    tok = tokenizer(
        subseqs,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_len,
    ).to(DEVICE)

    layer_weights = torch.tensor([0.1, 0.2, 0.3, 0.4], device=DEVICE).view(4, 1, 1, 1)

    # AMP autocast
    if AMP_DTYPE is not None and DEVICE == "cuda":
        autocast_ctx = torch.autocast(device_type="cuda", dtype=AMP_DTYPE)
    else:
        from contextlib import nullcontext
        autocast_ctx = nullcontext()

    with autocast_ctx:
        out = model(**tok, output_hidden_states=True)
        # out.hidden_states: tuple(L+1) including embeddings; we take last 4
        hs = torch.stack(out.hidden_states[-4:], dim=0)  # (4, B, T, H)
        weighted = (hs * layer_weights).sum(0)           # (B, T, H)

        mask = tok["attention_mask"].unsqueeze(-1)       # (B, T, 1)
        emb = (weighted * mask).sum(1) / mask.sum(1).clamp(min=1)  # (B, H)

    # view 평균
    emb_mean = emb.mean(0, keepdim=True).cpu()  # (1, H)
    return emb_mean

# ---------------------------------------------------
# 6️⃣ 전체 배치 임베딩 추출
#    (여전히 row-by-row지만 per-row에서 2*n_views 병렬 처리로 GPU utilization↑)
# ---------------------------------------------------
all_ids = []
all_embs = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    emb = get_seq_embedding(row["seq"], model, tokenizer, n_views=6, max_len=1024)
    all_ids.append(row["ID"])
    all_embs.append(emb)

emb_tensor = torch.vstack(all_embs)  # (N, H)
print("✅ Embedding tensor shape:", emb_tensor.shape)

# ---------------------------------------------------
# 7️⃣ LayerNorm + PCA Whitening
# ---------------------------------------------------
from torch.nn.functional import layer_norm

emb_normed = layer_norm(emb_tensor, emb_tensor.shape[1:])
emb_np = emb_normed.numpy()

# PCA Whitening (512차원 축소 + 정규화)
pca = PCA(n_components=min(512, emb_np.shape[1]), whiten=True, random_state=SEED)
emb_pca = pca.fit_transform(emb_np)

# 다시 정규화
emb_final = emb_pca / np.linalg.norm(emb_pca, axis=1, keepdims=True)
print("✅ Final embedding shape:", emb_final.shape)

# ---------------------------------------------------
# 8️⃣ 제출 파일 생성
# ---------------------------------------------------
emb_cols = [f"emb_{i:04d}" for i in range(emb_final.shape[1])]
emb_df = pd.DataFrame(emb_final, columns=emb_cols)
submission = pd.concat([pd.Series(all_ids, name="ID"), emb_df], axis=1)

out_path = "/content/submission_final_gLM2.csv"
submission.to_csv(out_path, index=False)
print("✅ Saved submission file:", out_path)

# ---------------------------------------------------
# 9️⃣ 로컬 다운로드
# ---------------------------------------------------
from google.colab import files
files.download(out_path)


In [1]:
# ============================================
#  제2회 MAI 대회 최종 제출 Inference (Stage2 LoRA 우선)
#  Strategy: Weighted last4 + Multi-crop + Reverse complement + PCA Whitening + LayerNorm
# ============================================

import os, math, numpy as np, pandas as pd, torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModelForMaskedLM, BitsAndBytesConfig

# (옵션) PEFT 로더: LoRA 체크포인트가 있을 때 사용
try:
    from peft import AutoPeftModelForMaskedLM
    HAS_PEFT = True
except Exception:
    HAS_PEFT = False

# -----------------------------
# 0) 환경설정
# -----------------------------
SEED = 42
np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", DEVICE)

# 데이터 경로
data_path = "/content/open"  # test.csv, sample_submission.csv가 있는 폴더
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
sub_df  = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
print("✅ Loaded:", test_df.shape, sub_df.shape)

# 모델/체크포인트 경로
MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"
STAGE2_CKPT = "/content/stage2_contrastive_lora8bit"  # 이전 파인튜닝에서 저장한 경로(없으면 자동 폴백)

# Inference 하이퍼파라미터
N_VIEWS   = 3      # multi-crop views per strand (원본/역상보 각각)
MAX_LEN   = 512    # crop 길이 (제출은 1024여도 OK; 여기선 안정성과 속도 균형)
N_PCA     = 512    # 최종 차원 (<= 2048)

# Layer 가중치 (마지막 4개 레이어)
LAYER_WEIGHTS = torch.tensor([0.1, 0.2, 0.3, 0.4])

# -----------------------------
# 1) reverse complement
# -----------------------------
def reverse_complement(seq: str) -> str:
    tr = str.maketrans("ACGT", "TGCA")
    return seq.translate(tr)[::-1]

# -----------------------------
# 2) 모델/토크나이저 로드 (Stage2 우선, 폴백은 베이스)
# -----------------------------
bnb_config = BitsAndBytesConfig(load_in_8bit=True)  # 8bit 로 VRAM 절약 (원한다면 FP16로 바꿔도 됨)

if os.path.isdir(STAGE2_CKPT) and HAS_PEFT:
    print("✅ Loading Stage2 LoRA checkpoint:", STAGE2_CKPT)
    tokenizer = AutoTokenizer.from_pretrained(STAGE2_CKPT, trust_remote_code=True)
    model = AutoPeftModelForMaskedLM.from_pretrained(
        STAGE2_CKPT, trust_remote_code=True,
        quantization_config=bnb_config, device_map={"": 0}
    )
else:
    # 폴백: 베이스 모델 사용
    print("⚠️ Stage2 ckpt not found or PEFT unavailable. Falling back to base model.")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    model = AutoModelForMaskedLM.from_pretrained(
        MODEL_ID, trust_remote_code=True,
        quantization_config=bnb_config, device_map={"": 0}
    )

model.eval()
model.config.use_cache = False
LAYER_WEIGHTS = LAYER_WEIGHTS.to(model.device)

# -----------------------------
# 3) 시퀀스 임베딩 함수
# -----------------------------
@torch.no_grad()
def embed_one_sequence(seq: str, n_views=N_VIEWS, max_len=MAX_LEN):
    """원본 + 역상보에 대해 랜덤 crop n_views씩 -> 총 2*n_views view 평균"""
    embs = []
    strands = [seq, reverse_complement(seq)]
    for s in strands:
        L = len(s)
        for _ in range(n_views):
            if L <= max_len:
                sub_seq = s
            else:
                start = np.random.randint(0, L - max_len + 1)
                sub_seq = s[start:start + max_len]

            tok = tokenizer(
                sub_seq, return_tensors="pt",
                truncation=True, padding=True, max_length=max_len
            ).to(model.device)

            out = model(**tok, output_hidden_states=True)
            hs  = torch.stack(out.hidden_states[-4:], dim=0)              # (4, B, L, H)
            w   = (hs * LAYER_WEIGHTS.view(4,1,1,1)).sum(0)               # (B, L, H) weighted sum
            mask = tok["attention_mask"].unsqueeze(-1)                    # (B, L, 1)
            emb  = (w * mask).sum(1) / mask.sum(1).clamp(min=1)           # (B, H)
            embs.append(emb.squeeze(0).cpu())

    return torch.stack(embs).mean(0)  # (H,)

# -----------------------------
# 4) 전체 추론 (단순 루프; 안정성 우선)
#    * 더 빠르게 하려면 배치화 가능하나 multi-crop/RC 동기화가 복잡해져서 여기선 루프 권장
# -----------------------------
all_ids, all_embs = [], []
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Embedding"):
    emb = embed_one_sequence(row["seq"])
    all_ids.append(row["ID"])
    all_embs.append(emb)

emb_tensor = torch.vstack(all_embs)           # (N, H)
print("✅ Raw embedding shape:", emb_tensor.shape)

# -----------------------------
# 5) 정규화 + Whitening (PCA)
# -----------------------------
from torch.nn.functional import layer_norm
emb_normed = layer_norm(emb_tensor, emb_tensor.shape[1:])   # (N, H)
emb_np = emb_normed.numpy()

# PCA 차원 축소 + 화이트닝
n_comp = min(N_PCA, emb_np.shape[1])
pca = PCA(n_components=n_comp, whiten=True, random_state=SEED)
emb_pca = pca.fit_transform(emb_np)                          # (N, n_comp)

# L2 normalize (cosine 안정화)
emb_final = emb_pca / (np.linalg.norm(emb_pca, axis=1, keepdims=True) + 1e-9)
print("✅ Final embedding shape:", emb_final.shape)

# -----------------------------
# 6) 제출 파일 생성
# -----------------------------
assert emb_final.shape[1] <= 2048, "임베딩 차원 2048 초과 금지!"
emb_cols = [f"emb_{i:04d}" for i in range(emb_final.shape[1])]
emb_df = pd.DataFrame(emb_final, columns=emb_cols)
submission = pd.concat([pd.Series(all_ids, name="ID"), emb_df], axis=1)

out_path = "/content/submission_final_stage2_or_base.csv"
submission.to_csv(out_path, index=False)
print("🎯 Saved:", out_path)

# (선택) 바로 다운로드
try:
    from google.colab import files
    files.download(out_path)
except Exception as e:
    print("ℹ️ Colab files.download 실패 시, 왼쪽 Files 패널에서 직접 다운로드하세요.")
    print("Error:", e)


KeyboardInterrupt: 

In [1]:
# ============================================
#  제2회 MAI 대회 최종 제출 Inference (Stage2 LoRA 우선) - 고GPU 활용 버전
#  Strategy: Weighted last4 + Multi-crop(view-batch) + Reverse complement
#            + LayerNorm + PCA Whitening
#  Notes:
#    - 샘플×뷰 동시 배치로 forward → GPU 사용률↑
#    - FP16(권장, VRAM 여유시) or 8bit(메모리 절약)
#    - TF32, cuDNN benchmark, AMP 적용
# ============================================

import os, numpy as np, pandas as pd, torch
from tqdm import tqdm
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModelForMaskedLM, BitsAndBytesConfig

# (옵션) PEFT 로더: LoRA가 있을 때 사용
try:
    from peft import AutoPeftModelForMaskedLM
    HAS_PEFT = True
except Exception:
    HAS_PEFT = False

# -----------------------------
# 0) 환경설정
# -----------------------------
SEED = 42
np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", DEVICE)

# GPU 커널/정밀도 최적화
if DEVICE == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

def _amp_dtype():
    if DEVICE == "cuda" and torch.cuda.is_available():
        cap = torch.cuda.get_device_capability()
        return torch.bfloat16 if cap[0] >= 8 else torch.float16  # Ampere+ → bf16
    return None

AMP_DTYPE = _amp_dtype()

# -----------------------------
# 1) 경로/하이퍼파라미터
# -----------------------------
data_path = "/content/open"  # test.csv, sample_submission.csv
MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"
STAGE2_CKPT = "/content/stage2_contrastive_lora8bit"  # LoRA 경로(없으면 폴백)

# === 핵심 성능 파라미터 ===
USE_8BIT   = False   # ✅ VRAM 여유되면 False로 두고 FP16 추론 권장(속도↑/점유율↑)
BATCH_SEQ  = 32      # 한 번에 처리할 샘플 수 (VRAM 따라 16~64 튜닝)
N_VIEWS    = 4       # per strand views (원본/역상보 각각)
MAX_LEN    = 1024    # crop 길이 (512→1024로 늘리면 계산량↑)
N_PCA      = 512     # 최종 출력 차원(≤2048)

LAYER_WEIGHTS = torch.tensor([0.1, 0.2, 0.3, 0.4])  # 마지막 4개 레이어 가중치

# -----------------------------
# 2) 데이터 로드
# -----------------------------
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
sub_df  = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
print("✅ Loaded:", test_df.shape, sub_df.shape)

# -----------------------------
# 3) reverse complement
# -----------------------------
def reverse_complement(seq: str) -> str:
    tr = str.maketrans("ACGT", "TGCA")
    return seq.translate(tr)[::-1]

# -----------------------------
# 4) 모델/토크나이저 로드 (Stage2 우선)
# -----------------------------
if USE_8BIT:
    bnb_config = BitsAndBytesConfig(load_in_8bit=True)

    if os.path.isdir(STAGE2_CKPT) and HAS_PEFT:
        print("✅ Loading Stage2 LoRA (8bit):", STAGE2_CKPT)
        tokenizer = AutoTokenizer.from_pretrained(STAGE2_CKPT, trust_remote_code=True)
        model = AutoPeftModelForMaskedLM.from_pretrained(
            STAGE2_CKPT, trust_remote_code=True,
            quantization_config=bnb_config, device_map={"": 0}
        )
    else:
        print("⚠️ Stage2 ckpt not found or PEFT unavailable. Using base (8bit).")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
        model = AutoModelForMaskedLM.from_pretrained(
            MODEL_ID, trust_remote_code=True,
            quantization_config=bnb_config, device_map={"": 0}
        )
else:
    # FP16 로드 (권장)
    if os.path.isdir(STAGE2_CKPT) and HAS_PEFT:
        print("✅ Loading Stage2 LoRA (FP16):", STAGE2_CKPT)
        tokenizer = AutoTokenizer.from_pretrained(STAGE2_CKPT, trust_remote_code=True)
        model = AutoPeftModelForMaskedLM.from_pretrained(
            STAGE2_CKPT, trust_remote_code=True,
            torch_dtype=torch.float16, device_map={"": 0}
        )
    else:
        print("⚠️ Stage2 ckpt not found or PEFT unavailable. Using base (FP16).")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
        model = AutoModelForMaskedLM.from_pretrained(
            MODEL_ID, trust_remote_code=True,
            torch_dtype=torch.float16, device_map={"": 0}
        )

model.eval(); model.config.use_cache = False
LAYER_WEIGHTS = LAYER_WEIGHTS.to(model.device)

# -----------------------------
# 5) 뷰 생성 유틸
# -----------------------------
def make_crops(seq, n_views, max_len):
    out = []
    for s in (seq, reverse_complement(seq)):
        L = len(s)
        for _ in range(n_views):
            if L <= max_len:
                out.append(s)
            else:
                st = np.random.randint(0, L - max_len + 1)
                out.append(s[st:st+max_len])
    return out  # len = 2*n_views

# -----------------------------
# 6) 샘플×뷰 동시 배치 임베딩
# -----------------------------
from contextlib import nullcontext

@torch.no_grad()
def embed_batch(seq_list, n_views=N_VIEWS, max_len=MAX_LEN):
    """
    seq_list: 길이 B의 시퀀스 리스트
    - 각 시퀀스에 대해 (원본/역상보)×n_views crop 생성
    - 전체(2*n_views*B) 를 한 방에 토크나이즈 → 1 forward
    - 마지막 4개 레이어 가중합 → attention mask 평균 → 시퀀스별 view 평균
    반환: (B, H) torch.FloatTensor(cpu)
    """
    # 1) 모든 샘플의 crop 생성 후 평탄화
    batch_crops = []
    starts = [0]
    for seq in seq_list:
        cs = make_crops(seq, n_views, max_len)
        batch_crops.extend(cs)
        starts.append(starts[-1] + len(cs))   # 인덱스 경계 기록

    # 2) 한 번에 토크나이즈
    tok = tokenizer(
        batch_crops, return_tensors="pt",
        truncation=True, padding=True, max_length=max_len
    ).to(model.device)

    # 3) AMP (8bit일 땐 실효 적을 수 있어 nullcontext)
    use_amp = (not USE_8BIT) and (DEVICE == "cuda")
    autocast_ctx = torch.autocast("cuda", dtype=AMP_DTYPE) if use_amp else nullcontext()

    with autocast_ctx:
        out = model(**tok, output_hidden_states=True)
        hs  = torch.stack(out.hidden_states[-4:], dim=0)                 # (4,B,T,H)
        w   = (hs * LAYER_WEIGHTS.view(4,1,1,1)).sum(0)                  # (B,T,H)
        mask = tok["attention_mask"].unsqueeze(-1)                       # (B,T,1)
        emb  = (w * mask).sum(1) / mask.sum(1).clamp(min=1)              # (B,H)

    # 4) 원 시퀀스 단위로 view 평균
    outs = []
    for i in range(len(seq_list)):
        a, b = starts[i], starts[i+1]
        outs.append(emb[a:b].mean(0, keepdim=True))
    return torch.vstack(outs).cpu()                                      # (B, H)

# -----------------------------
# 7) 전체 추론 루프 (샘플 배치 단위)
# -----------------------------
all_ids, all_embs = [], []
seqs = test_df["seq"].tolist()
ids  = test_df["ID"].tolist()

for s in tqdm(range(0, len(seqs), BATCH_SEQ), desc="Embedding (samples×views batched)"):
    chunk = seqs[s:s+BATCH_SEQ]
    embs  = embed_batch(chunk)           # (B, H)
    all_embs.append(embs)
    all_ids.extend(ids[s:s+BATCH_SEQ])

emb_tensor = torch.vstack(all_embs)      # (N, H)
print("✅ Raw embedding shape:", emb_tensor.shape)

# -----------------------------
# 8) 정규화 + Whitening (PCA)
# -----------------------------
from torch.nn.functional import layer_norm
emb_normed = layer_norm(emb_tensor, emb_tensor.shape[1:])   # (N, H)
emb_np = emb_normed.numpy()

# PCA 차원 축소 + 화이트닝
n_comp = min(N_PCA, emb_np.shape[1])
pca = PCA(n_components=n_comp, whiten=True, random_state=SEED)
emb_pca = pca.fit_transform(emb_np)                          # (N, n_comp)

# L2 normalize (cosine 안정화)
emb_final = emb_pca / (np.linalg.norm(emb_pca, axis=1, keepdims=True) + 1e-9)
print("✅ Final embedding shape:", emb_final.shape)

# -----------------------------
# 9) 제출 파일 생성
# -----------------------------
assert emb_final.shape[1] <= 2048, "임베딩 차원 2048 초과 금지!"
emb_cols = [f"emb_{i:04d}" for i in range(emb_final.shape[1])]
emb_df = pd.DataFrame(emb_final, columns=emb_cols)
submission = pd.concat([pd.Series(all_ids, name="ID"), emb_df], axis=1)

out_path = "/content/submission_final_stage2_or_base.csv"
submission.to_csv(out_path, index=False)
print("🎯 Saved:", out_path)

# (선택) Colab 다운로드
try:
    from google.colab import files
    files.download(out_path)
except Exception as e:
    print("ℹ️ files.download 실패 시, 왼쪽 Files 패널에서 직접 다운로드하세요.")
    print("Error:", e)


✅ Device: cuda
✅ Loaded: (13711, 2) (13711, 769)
⚠️ Stage2 ckpt not found or PEFT unavailable. Falling back to base model.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Embedding(batched views):   0%|          | 0/13711 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torc

KeyboardInterrupt: 